# Spherocylinders in HOOMD

## Overview

### Questions

* How do I run simulations of hard spherocylinders in HOOMD?

### Objectives

* Highlight the code changes needed to convert the hard sphere simulation to model hard spherocylinders.

In [ ]:
# This cell installs HOOMD in Google Colab. Delete it if you run locally
!pip install -q condacolab
import condacolab
condacolab.install_from_url('https://github.com/joaander/hoomd-che629/releases/download/2022.0.0-beta.1/hoomd-che629-2022.0-Linux-x86_64.sh')

## Boilerplate code

In [ ]:
import hoomd
import math
import itertools
import numpy
import copy
import gsd.hoomd
import freud
import matplotlib
import IPython
import rowan
%matplotlib inline
matplotlib.style.use('ggplot')

The `render` function in the next (hidden) cell will render a snapshot using **fresnel**.

<div class="alert alert-info">
    This is not intended as a full tutorial on <b>fresnel</b> - see the <a href="https://fresnel.readthedocs.io/">fresnel user documentation</a> if you would like to learn more.
</div>

In [ ]:
import fresnel

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=250, h=250)

def render(snapshot, params):
    box_L = snapshot.configuration.box[0]

    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Cylinder(scene,
                                         N=snapshot.particles.N,
                                         radius=params['D']/2)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([252/255, 209/255, 1/255]),
                                                  roughness=0.5)
    
    top = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,params['L']/2])
    bottom = snapshot.particles.position[:] + rowan.rotate(snapshot.particles.orientation, [0,0,-params['L']/2])
    
    geometry.points[:,0,:] = top
    geometry.points[:,1,:] = bottom
    geometry.outline_width = 0.05
    box = fresnel.geometry.Box(scene, snapshot.configuration.box, box_radius=.04)
    
    scene.lights = [fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
                    fresnel.light.Light(direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3)]
    scene.camera = fresnel.camera.Orthographic(position=(snapshot.configuration.box[0], -snapshot.configuration.box[1]-1, snapshot.configuration.box[2]),
                                               look_at=(0, 0, 0),
                                               up=(0, 0, 1),
                                               height=1.2*(box_L+params['L']))
    scene.background_color = (1,1,1)
    return tracer.sample(scene, samples=100)

## Spherocylinders

You can describe a spheroclyinder with a length *L* between end points and the diameter *D*.
Place *N_particles* particles in the at a density of $\rho^*$. $\rho^*$ is the volume fraction relative to the densest possible packing (in the range 0-1) and is commonly used in the spheroclyinder literature.

In [ ]:
L=5
D=1
N_particles = 50
rho_star = 0.53

You can represent a spherocylinder as a 2-vertex spheropolyhedron.

In [ ]:
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)

## Setting the initial condition

The spherical caps extend past the end points, so end-to-end spherocylinders must be placed at least a distance of L+D apart.

In [ ]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles

spacing = D * 2
K = math.ceil(N_particles**(1/2))
box_L = K * spacing
x = numpy.linspace(-box_L / 2, box_L / 2, K, endpoint=False)
position_2d = list(itertools.product(x, repeat=2))
position_2d = position_2d[0:N_particles]

In [ ]:
snapshot.particles.position = numpy.zeros(shape=(N_particles, 3))
snapshot.particles.position[:,0:2] = position_2d
snapshot.particles.orientation = [1,0,0,0]*N_particles
snapshot.particles.types = ['spherocylinder']
snapshot.configuration.box = [box_L, box_L, 2.0*(box_L + D), 0, 0, 0]

with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
    f.append(snapshot)

In [ ]:
render(snapshot, dict(L=L, D=D))

## Randomizing the system

As with the hard spheres, run the simulation to randomize the particle positions and orientations.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='lattice.gsd')
sim.run(10e3)
hoomd.write.GSD.write(state=sim.state, filename='random.gsd')

In [ ]:
render(sim.state.get_snapshot(), dict(L=L, D=D))

## Compressing the system

Use **QuickCompress** to compress the simulation to the target density.
Differences compared to the hard sphere compression code: Computation of the final box, **MoveSize** applies to 'a' moves as well as 'd'.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='random.gsd')

In [ ]:
rho_c = 2/(math.sqrt(2) + (L/D)*math.sqrt(3))
rho = rho_c * rho_star
box_V = sim.state.N_particles / rho   
box_L=box_V**(1/3)
final_box = hoomd.Box.cube(box_L)

compress = hoomd.hpmc.update.QuickCompress(trigger=hoomd.trigger.Periodic(10),
                                           target_box = final_box)
sim.operations.updaters.append(compress)

tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=hoomd.trigger.Periodic(10),
                                             max_translation_move=0.2,
                                             max_rotation_move=0.2)
sim.operations.tuners.append(tune)   

while not compress.complete and sim.timestep < 5e4:
    sim.run(1000)

hoomd.write.GSD.write(state=sim.state, filename='compressed.gsd')

In [ ]:
render(sim.state.get_snapshot(), dict(L=L, D=D))

## Equilibrating the system

Tune both 'a' and 'd' move sizes and run the simulation for 200,000 steps.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=18)
mc = hoomd.hpmc.integrate.ConvexSpheropolyhedron()
sim.operations.integrator = mc
mc.shape['spherocylinder'] = dict(vertices=[[0,0,-L/2],
                                            [0,0,L/2]],
                                  sweep_radius=D/2)
sim.create_state_from_gsd(filename='compressed.gsd')

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb')
sim.operations.writers.append(gsd_writer)

tune = hoomd.hpmc.tune.MoveSize.scale_solver(moves=['a', 'd'],
                                             target=0.2,
                                             trigger=hoomd.trigger.And(
                                                 [hoomd.trigger.Periodic(100),
                                                  hoomd.trigger.Before(sim.timestep + 5000)]))
sim.operations.tuners.append(tune)   

logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities=['timestep', 'final_timestep', 'tps'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)
sim.operations.writers.append(table)

sim.run(200e3)

## Analyze the results

Visualize the final state of the simulation.

In [ ]:
render(sim.state.get_snapshot(), dict(L=L, D=D))

Compute the nematic order in each frame of the trajectory.

In [ ]:
with gsd.hoomd.open('trajectory.gsd') as traj:
    nematic = freud.order.Nematic([0, 0, 1])
    nematic_order = []
    for frame in traj:
        nematic.compute(frame.particles.orientation)
        nematic_order.append(nematic.order)

fig = matplotlib.figure.Figure(figsize=(10, 6.18))
ax = fig.add_subplot()
ax.plot(nematic_order)
ax.set_xlabel('frame')
ax.set_ylabel('average nematic order parameter')
ax.set_ylim([0, 1]);

In [ ]:
fig

[Previous section](00-index.ipynb). [Next section](../02-Homework/00-index.ipynb).